In [1]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\korek\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
import re
from datasets import load_dataset
import numpy as np
import pandas as  pd
from pprint import pprint# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel# spaCy for preprocessing
import spacy# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [4]:
# LoadDataset
# df=pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
# print(df.target_names.unique())
# df.head()
dataset = load_dataset("squad_v2", split="train")
print(dataset)

['rec.autos' 'comp.sys.mac.hardware' 'comp.graphics' 'sci.space'
 'talk.politics.guns' 'sci.med' 'comp.sys.ibm.pc.hardware'
 'comp.os.ms-windows.misc' 'rec.motorcycles' 'talk.religion.misc'
 'misc.forsale' 'alt.atheism' 'sci.electronics' 'comp.windows.x'
 'rec.sport.hockey' 'rec.sport.baseball' 'soc.religion.christian'
 'talk.politics.mideast' 'talk.politics.misc' 'sci.crypt']


Found cached dataset squad_v2 (C:/Users/korek/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 130319
})


In [5]:
dataset["question"][:10]

['When did Beyonce start becoming popular?',
 'What areas did Beyonce compete in when she was growing up?',
 "When did Beyonce leave Destiny's Child and become a solo singer?",
 'In what city and state did Beyonce  grow up? ',
 'In which decade did Beyonce become famous?',
 'In what R&B group was she the lead singer?',
 'What album made her a worldwide known artist?',
 "Who managed the Destiny's Child group?",
 'When did Beyoncé rise to fame?',
 "What role did Beyoncé have in Destiny's Child?"]

In [6]:
# Convert to list 
# data = df.content.values.tolist()  
data = dataset["question"]
# Remove Emails 
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]  
# Remove new line characters 
data = [re.sub('\s+', ' ', sent) for sent in data]  
# Remove distracting single quotes 
data = [re.sub("\'", "", sent) for sent in data]  
pprint(data[:1])

<>:5: DeprecationWarning: invalid escape sequence \S
<>:7: DeprecationWarning: invalid escape sequence \s
C:\Users\korek\AppData\Local\Temp/ipykernel_1936/3380574120.py:5: DeprecationWarning: invalid escape sequence \S
  data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]
C:\Users\korek\AppData\Local\Temp/ipykernel_1936/3380574120.py:7: DeprecationWarning: invalid escape sequence \s
  data = [re.sub('\s+', ' ', sent) for sent in data]


['When did Beyonce start becoming popular?']


In [7]:
def sent_to_words(sentences):
  for sentence in sentences:
    yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))            #deacc=True removes punctuations
data_words = list(sent_to_words(data))
print(data_words[:1])

[['when', 'did', 'beyonce', 'start', 'becoming', 'popular']]


In [8]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)
# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['when', 'did', 'beyonce', 'start', 'becoming', 'popular']


In [9]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [10]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

C:\Python38\lib\site-packages\spacy\language.py:1895: UserWarning: [W123] Argument disable with value ['parser', 'ner'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(


[['beyonce', 'start', 'become', 'popular']]


In [11]:
# Create Dictionary 
id2word = corpora.Dictionary(data_lemmatized)  
# Create Corpus 
texts = data_lemmatized  
# Term Document Frequency 
corpus = [id2word.doc2bow(text) for text in texts]  
# View 
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1)]]


In [12]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [13]:
# Print the keyword of topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.193*"new" + 0.168*"occur" + 0.141*"include" + 0.062*"remove" + '
  '0.051*"recognize" + 0.039*"film" + 0.025*"top" + 0.018*"criticize" + '
  '0.000*"matter" + 0.000*"genocide"'),
 (1,
  '0.259*"make" + 0.187*"begin" + 0.171*"know" + 0.082*"major" + 0.044*"bring" '
  '+ 0.021*"enter" + 0.016*"artist" + 0.014*"worldwide" + 0.013*"child" + '
  '0.013*"hit"'),
 (2,
  '0.535*"first" + 0.144*"act" + 0.000*"police" + 0.000*"genocide" + '
  '0.000*"much" + 0.000*"land" + 0.000*"establish" + 0.000*"develop" + '
  '0.000*"war" + 0.000*"use"'),
 (3,
  '0.372*"become" + 0.095*"person" + 0.083*"change" + 0.055*"industry" + '
  '0.052*"woman" + 0.037*"declare" + 0.013*"mother" + 0.000*"percentage" + '
  '0.000*"hunt" + 0.000*"animal"'),
 (4,
  '0.236*"area" + 0.112*"believe" + 0.099*"grow" + 0.074*"fight" + 0.074*"age" '
  '+ 0.073*"leader" + 0.069*"receive" + 0.023*"competition" + 0.000*"hunt" + '
  '0.000*"use"'),
 (5,
  '0.267*"long" + 0.177*"help" + 0.112*"old" + 0.080*"follow" + 0.050

In [14]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  
# a measure of how good the model is. lower the better.

# Compute Coherence Score
# coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
# coherence_lda = coherence_model_lda.get_coherence()
# print('\nCoherence Score: ', coherence_lda)


Perplexity:  -26.3637413221724


In [15]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

C:\Python38\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
13     0.251465  0.314372       1        1  11.307137
15     0.185782 -0.042470       2        1  10.792841
8      0.232138 -0.289592       3        1   9.350827
1      0.108954 -0.010121       4        1   8.579330
14     0.006062  0.000016       5        1   6.203422
10     0.061806 -0.003954       6        1   6.035599
12    -0.006636  0.000644       7        1   5.322671
19     0.004152  0.000116       8        1   5.203082
17    -0.009071  0.000756       9        1   4.919596
4     -0.012393  0.000908      10        1   4.780686
0     -0.057280  0.002513      11        1   4.532837
7     -0.043486  0.002097      12        1   4.314211
3     -0.051921  0.002357      13        1   4.268586
5     -0.051331  0.002339      14        1   3.988866
2     -0.064673  0.002707      15        1   3.702134
11    -0.108373  0.003463      16        1   2.220799
18    -0.107213  0.003457      17        1   1.288411
9     -0.110313  0.003468      18        1   1.193077
6     -0.115519  0.003456      19        1   1.053712
16    -0.112148  0.003468      20        1   0.942176, topic_info=            Term          Freq         Total Category  logprob  loglift
30          many  40627.000000  40627.000000  Default  30.0000  30.0000
82          year  28548.000000  28548.000000  Default  29.0000  29.0000
125         type  19224.000000  19224.000000  Default  28.0000  28.0000
25         first  10540.000000  10540.000000  Default  27.0000  27.0000
12         state  11311.000000  11311.000000  Default  26.0000  26.0000
...          ...           ...           ...      ...      ...      ...
1118      hunter      0.112604      4.412186  Topic20 -10.7043   0.9965
2849   christian      0.112600      4.410575  Topic20 -10.7043   0.9968
27303  kathmandu      0.112605      4.412944  Topic20 -10.7043   0.9963
4157     hunting      0.112602      4.411606  Topic20 -10.7043   0.9966
2512      police      0.112605      4.413233  Topic20 -10.7043   0.9963

[831 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
56       15  0.998357        act
126      10  0.997809        age
16       16  0.347433      album
85        8  0.997600       also
167      12  0.990474     annual
...     ...       ...        ...
53       12  0.999187       work
81        2  0.999526      world
20        4  0.993775  worldwide
82        2  0.999851       year
89        4  0.985791      young

[124 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[14, 16, 9, 2, 15, 11, 13, 20, 18, 5, 1, 8, 4, 6, 3, 12, 19, 10, 7, 17])